In [1]:
# Imports

import pandas as pd
from pandas import Series,DataFrame
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

Load titanic train and test csv files

In [3]:
X_train = pd.read_csv("./train.csv")
y_train = X_train.pop("Survived")

X_test_orig = pd.read_csv("./test.csv")

In [4]:
data = X_train.append(X_test_orig, ignore_index=True)

In [5]:
data['Title'] = data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
data.Title.value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Major             2
Ms                2
Mlle              2
Dona              1
Capt              1
Sir               1
Jonkheer          1
Lady              1
the Countess      1
Don               1
Mme               1
Name: Title, dtype: int64

In [6]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
data.Title = data.Title.map(normalized_titles)
data.Title.value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64

In [7]:
# group by Sex, Pclass, and Title
grouped = data.groupby(['Sex','Pclass', 'Title'])
grouped.Age.median()
data.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

In [8]:
data['CatAge'] = pd.qcut(data.Age, q=5, labels=False )

In [9]:
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1

In [10]:
data["Fare"].fillna(data.Fare.mean(), inplace=True)

most_embarked = data.Embarked.value_counts().index[0]
data["Embarked"].fillna(most_embarked, inplace=True)
data.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
Title             0
CatAge            0
Family_Size       0
dtype: int64

In [11]:
data.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Title           object
CatAge           int64
Family_Size      int64
dtype: object

In [12]:
data = pd.get_dummies(data, columns=['Sex'], prefix = ['Sex'])
data = pd.get_dummies(data, columns=['Embarked'], prefix = ['Embarked'])
data = pd.get_dummies(data, columns=['Title'], prefix = ['Title'])

In [13]:
# drop columns won't be useful in analysis and prediction
data = data.drop(['PassengerId','Name','Ticket', 'Cabin', 'Age', 'SibSp', 'Parch'], axis=1)

In [14]:
data.head()

,Pclass,Fare,CatAge,Family_Size,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,3,7.2500,1,2,0,1,0,0,1,0,0,1,0,0,0
1,1,71.2833,3,2,1,0,1,0,0,0,0,0,1,0,0
2,3,7.9250,1,1,1,0,0,0,1,0,1,0,0,0,0
3,1,53.1000,3,2,1,0,0,0,1,0,0,0,1,0,0
4,3,8.0500,3,1,0,1,0,0,1,0,0,1,0,0,0


In [15]:
numeric_variables = list(data.dtypes[data.dtypes != "object"].index)
data = data[numeric_variables]

In [16]:
data.isnull().sum()

Pclass           0
Fare             0
CatAge           0
Family_Size      0
Sex_female       0
Sex_male         0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
Title_Master     0
Title_Miss       0
Title_Mr         0
Title_Mrs        0
Title_Officer    0
Title_Royalty    0
dtype: int64

construct the X_Train and X_Test

In [17]:
X_train = data.iloc[:891]
X_test = data.iloc[891:]

In [18]:
data.dtypes

Pclass             int64
Fare             float64
CatAge             int64
Family_Size        int64
Sex_female         uint8
Sex_male           uint8
Embarked_C         uint8
Embarked_Q         uint8
Embarked_S         uint8
Title_Master       uint8
Title_Miss         uint8
Title_Mr           uint8
Title_Mrs          uint8
Title_Officer      uint8
Title_Royalty      uint8
dtype: object

### Modeling - Decision Tree Classifier GridSearch CV

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
tree_clf = DecisionTreeClassifier()
dt_params =[
            {'max_depth': [1, 2, 4, 8, 16, 32, 64], 
             'min_samples_leaf': [1, 2, 3, 4, 5, 6],
             },
]

In [20]:
tree_clf_cv = GridSearchCV(estimator=tree_clf, param_grid=dt_params, cv=4)
tree_clf_cv.fit(X_train, y_train)

GridSearchCV(cv=4, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [1, 2, 4, 8, 16, 32, 64], 'min_samples_leaf': [1, 2, 3, 4, 5, 6]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
cross_val_score(tree_clf_cv.best_estimator_, X_train, y_train, cv=3, scoring="accuracy")

array([0.77441077, 0.84511785, 0.81481481])

In [22]:
cross_val_score(tree_clf_cv.best_estimator_, X_train, y_train, cv=3, scoring="precision")

array([0.74226804, 0.81481481, 0.84705882])

In [23]:
cross_val_score(tree_clf_cv.best_estimator_, X_train, y_train, cv=3, scoring="recall")

array([0.63157895, 0.77192982, 0.63157895])

In [24]:
print("Optimal params: {}".format(tree_clf_cv.best_estimator_))

Optimal params: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


### Decision Tree - ROC Curve

In [25]:
X_train_small = data.iloc[:800]
X_test_small = data.iloc[800:891]
y_train_small = y_train.iloc[:800]
y_test_small = y_train.iloc[800:891]

In [26]:
dt1 = DecisionTreeClassifier(random_state = 42)
dt1.fit(X_train_small, y_train_small)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [27]:
cross_val_score(dt1, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.78651685, 0.7752809 , 0.80451128])

In [28]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt1.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.79437564499484

### Decision Tree - ROC Curve with Fine Tuned Parameters

In [29]:
dt_cv = GridSearchCV(estimator=tree_clf, param_grid=dt_params, cv=4)
dt_cv.fit(X_train_small, y_train_small)

GridSearchCV(cv=4, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [1, 2, 4, 8, 16, 32, 64], 'min_samples_leaf': [1, 2, 3, 4, 5, 6]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
cross_val_score(dt_cv.best_estimator_, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.78277154, 0.85018727, 0.81954887])

In [31]:
cross_val_score(dt_cv.best_estimator_, X_train_small, y_train_small, cv=3, scoring="precision")

array([0.78481013, 0.77570093, 0.875     ])

In [32]:
cross_val_score(dt_cv.best_estimator_, X_train_small, y_train_small, cv=3, scoring="recall")

array([0.59223301, 0.82524272, 0.61764706])

In [33]:
print("Optimal params: {}".format(dt_cv.best_estimator_))

Optimal params: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [34]:
y_pred_small = dt_cv.best_estimator_.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.7915376676986585

Improves the score by fine tuning the parameters

### RBF Classifier

Added back from Week 7

In [35]:
svc_clf_2 =  SVC()
svc_clf_2.fit(X_train_small, y_train_small)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [36]:
cross_val_score(svc_clf_2, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.75655431, 0.80524345, 0.77443609])

In [37]:
y_pred_small = svc_clf_2.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.8503611971104231

Still the reigning champion!

### Polynomial Classifier

Added back from Week 7

In [38]:
svc_clf_3 =  SVC(kernel='poly', degree=2)
svc_clf_3.fit(X_train_small, y_train_small)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
cross_val_score(svc_clf_3, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.81273408, 0.83520599, 0.80451128])

In [40]:
y_pred_small = svc_clf_3.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.8444272445820433

### SVC GridSearchCV 

Added back from Week 7

In [41]:
svc_cv = GridSearchCV(estimator=tree_clf, param_grid=dt_params, cv=4)
svc_cv.fit(X_train_small, y_train_small)

GridSearchCV(cv=4, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [1, 2, 4, 8, 16, 32, 64], 'min_samples_leaf': [1, 2, 3, 4, 5, 6]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [42]:
cross_val_score(svc_cv, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.80149813, 0.83146067, 0.82706767])

In [43]:
y_pred_small = svc_cv.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.7827657378740971

### Voting Classifier 

In [44]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier (
    estimators=[('poly_svc', svc_clf_3),
                 ('rbf_svc_2', svc_clf_2),
                 ('best', svc_cv.best_estimator_)],
                 voting='hard')
voting_clf.fit(X_train_small, y_train_small)

VotingClassifier(estimators=[('poly_svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('rbf_svc_2', SVC(C=1.0, cache_size=200, class_...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [45]:
cross_val_score(voting_clf, X_train_small, y_train_small, cv=3, scoring="accuracy")

/Users/jsp22/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jsp22/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jsp22/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0.82022472, 0.83146067, 0.82706767])

In [46]:
y_pred_small = voting_clf.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

/Users/jsp22/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8444272445820433

Good score but not the best

### Random Forest Classifier

In [47]:
rnd_clf2 = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=1)
rnd_clf2.fit(X_train_small, y_train_small)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
cross_val_score(rnd_clf2, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.82022472, 0.84269663, 0.80827068])

In [49]:
importances = rnd_clf2.feature_importances_

In [50]:
print(importances)

[0.10123054 0.11848311 0.02898349 0.06967757 0.17571384 0.16715788
 0.01078831 0.00804333 0.01002917 0.0163668  0.06110666 0.18485302
 0.04115307 0.00507867 0.00133452]


In [51]:
feat_labels = data.columns
importances = rnd_clf2.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(15):
    print("%2d) %-*s %f" % (f + 1, 20, feat_labels[indices[f]], importances[indices[f]]))


 1) Title_Mr             0.184853
 2) Sex_female           0.175714
 3) Sex_male             0.167158
 4) Fare                 0.118483
 5) Pclass               0.101231
 6) Family_Size          0.069678
 7) Title_Miss           0.061107
 8) Title_Mrs            0.041153
 9) CatAge               0.028983
10) Title_Master         0.016367
11) Embarked_C           0.010788
12) Embarked_S           0.010029
13) Embarked_Q           0.008043
14) Title_Officer        0.005079
15) Title_Royalty        0.001335


In [52]:
y_pred_small = rnd_clf2.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.8444272445820433

### Adaboost Classifier

In [53]:
from sklearn.ensemble import AdaBoostClassifier
bdt_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=8, min_samples_leaf=4),
                             learning_rate=0.1,
                             n_estimators=5)
bdt_clf.fit(X_train_small, y_train_small)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.1, n_estimators=5, random_state=None)

In [54]:
cross_val_score(bdt_clf, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.77902622, 0.83146067, 0.81954887])

Not an improvement in cross validation scores

In [55]:
y_pred_small = bdt_clf.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.7974716202270382

I used the small train data sets, so that I could calculate ROC curves using the reserved test sets, to gain a better understanding the effectiveness of each model and avoid leaking.